This notebook walks through how to make graphs in this repo. You can copy this example.ipynb and rename it for your own graphing work. <b> Please do each graph in a seperate notebook!</b>

1. Import the appropriate packages. We'll use plotly.graph_objects for making all of our graphs.

In [1]:
import datetime as dt
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
import utils.utils as ut
import utils.graph_templates

from fredapi import Fred

2. We'll get data using the fredapi package. Set the path to the text file with your API key

In [2]:
# Call the graph the exact same thing as its notebook (minus the ipynb suffix) NAME FOR THE FILE, not the title
GRAPH_NAME = "job_opens_to_unemployed"

API_KEY_PATH = ut.get_repo_root() / "fred_api_key.txt" 

fred = Fred(api_key_file = API_KEY_PATH)

3. Set the fed_2025 template as default

In [3]:
pio.templates.default = 'fed_2025'

# Now is a good time to set the path to the graph output folder!
GRAPH_OUTPUT_PATH = ut.get_repo_root() / "figures"

4. Use the fredapi to get the data and prepare it for graphing. Documentation on the optional parameters that can be passed to the get_series called are found here (the documentation in fredapi is out of date). 

https://fred.stlouisfed.org/docs/api/fred/series_observations.html#Description

If you get data from somewhere else thats fine too! Put the raw csv in the "raw_data" folder and read it in here. Make sure not to edit the raw data, just transform and graph it.

In [4]:
# # The get_series_info method may be useful
# fred.get_series_info(series_id="CPIAUCSL")

# It's good practice to store the series codes in a dictionary with their names
series_codes = {
    "Job Openings: Total Nonfarm": "JTSJOL",
    "Unemployment Level": "UNEMPLOY",
}

In [7]:
today = dt.date.today()

jobs_srs = fred.get_series(
    series_id=series_codes["Job Openings: Total Nonfarm"],
    observation_start=dt.date(2024, 1, 1),
    observation_end=today,
    frequency='m',
).rename("Job Openings")

unemploy_srs = fred.get_series(
    series_id=series_codes["Unemployment Level"],
    observation_start=dt.date(2024, 1, 1),
    observation_end=today,
    frequency='m'
).rename("Unemployment Level")

# # Calculating job openings-to-umemployment ratio
# ratio=unemploy_srs/jobs_srs

# # a_ratio=ratio.round(2)

# # Naming the index makes it easier tot title
# a_ratio.index.name = "Date"

# a_ratio.tail()

joined_df = pd.concat(
    [jobs_srs, unemploy_srs], #u6_df], # srs to join
    axis=1, # multiple observations per index entry
    join='inner' # only include observations where none of the 3 are missing
)

joined_df.tail()

,Job Openings,Unemployment Level
2025-04-01,7395.0,7166.0
2025-05-01,7712.0,7237.0
2025-06-01,7357.0,7015.0
2025-07-01,7208.0,7236.0
2025-08-01,7227.0,7384.0


5. Now that all our data is ready, make the graph and have it save itself as a .html file to graph_output whenver the notebooks is rerun. The name of the file should exactly match the notebook name. For instance, this file "example.ipynb" produces the graph "example.html." Nice work, you made a graph! 

In [16]:
# First make the figure
fig = go.Figure()

# Loop the columns of the dataframe and plot each as a separate trace
for col in joined_df.columns:
    fig.add_trace(
        go.Scatter(
            x=joined_df.index,
            y=joined_df[col],
            mode='lines',
            name=col
        )
    )
# Update the titles, using the html tage <sup> for a subtitle 
fig.update_layout(
    title = dict(text = 'Job Seekers and Job Openings<br><sup>Monthly, Seasonally Adjusted </sup>'),
    xaxis_title="Date",
    # yaxis_title="Job Seekers  Job Opening",
)

# This is graph specific, but here we want the y-axis to be percent signs 
fig.update_yaxes(
    tickformat=".0f",
)

# Again, graph specific, we have a mutliyear series and want tick marks to be years
fig.update_xaxes(
    type='date',
    dtick="M4"
    )

fig.update_layout(showlegend=True,
                legened=dict(font=dict(size=20)),
                height=600,
                width=900,)

# ut.add_end_labels(fig = fig)


# fig.add_hline(
#     y=1,
#     line_dash="dot",
#     line_color="gray",
#     annotation_text="1 Job Seeker per Job Opening",
#     annotation_position="bottom right"
# )

fig.update_yaxes(
    title="Thousands"
)

# Show our figure (Dimensions may be off on different screen sizes)
fig.show()

# This should be the same for EVERY GRAPH!
# Save it to the graph_output folder with the name matching the file, as HTML
fig.write_html(GRAPH_OUTPUT_PATH / f"{GRAPH_NAME}.html")

ValueError: Invalid property specified for object of type plotly.graph_objs.Layout: 'legened'

Did you mean "legend"?

    Valid properties:
        activeselection
            :class:`plotly.graph_objects.layout.Activeselection`
            instance or dict with compatible properties
        activeshape
            :class:`plotly.graph_objects.layout.Activeshape`
            instance or dict with compatible properties
        annotations
            A tuple of
            :class:`plotly.graph_objects.layout.Annotation`
            instances or dicts with compatible properties
        annotationdefaults
            When used in a template (as
            layout.template.layout.annotationdefaults), sets the
            default property values to use for elements of
            layout.annotations
        autosize
            Determines whether or not a layout width or height that
            has been left undefined by the user is initialized on
            each relayout. Note that, regar